<a href="https://colab.research.google.com/github/quyettranvu/deep_learning_hands_on/blob/main/chapter_builders-guide/init-param.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter Initialization

Now that we know how to access the parameters,
let's look at how to initialize them properly.
We discussed the need for proper initialization in :numref:`sec_numerical_stability`.
The deep learning framework provides default random initializations to its layers.
However, we often want to initialize our weights
according to various other protocols. The framework provides most commonly
used protocols, and also allows to create a custom initializer.


In [1]:
import torch
from torch import nn

By default, PyTorch initializes weight and bias matrices
uniformly by drawing from a range that is computed according to the input and output dimension.
PyTorch's `nn.init` module provides a variety
of preset initialization methods.


In [12]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1)) # after forward, LazyLinear becomes Linear
X = torch.rand(size=(2, 4)) # 2 samples, 4 input features
print(f"Before materialization: {net.state_dict()}")
_ = net(X) # materialize LazyLinear, this will make the layer understand input size of weight, bias
print(f"After materialization: {net.state_dict()}")
net(X).shape
net[0].weight.data, net[0].bias.data

Before materialization: OrderedDict({'0.weight': <UninitializedParameter>, '0.bias': <UninitializedParameter>, '2.weight': <UninitializedParameter>, '2.bias': <UninitializedParameter>})
After materialization: OrderedDict({'0.weight': tensor([[-0.4547,  0.0774, -0.3541, -0.1512],
        [-0.0430,  0.1267,  0.0890,  0.3176],
        [-0.3321,  0.3767, -0.0680, -0.1455],
        [-0.3117,  0.0956, -0.4265, -0.2773],
        [ 0.2602, -0.4928, -0.1845,  0.2387],
        [ 0.4033,  0.4966, -0.3301, -0.2480],
        [-0.3596, -0.4486, -0.0939, -0.1304],
        [-0.0832,  0.1713, -0.4213, -0.2423]]), '0.bias': tensor([ 0.2074,  0.0152, -0.3750, -0.1547,  0.2195, -0.2412,  0.2158,  0.0358]), '2.weight': tensor([[-0.3049, -0.2203, -0.0625,  0.0117, -0.1791, -0.2451, -0.0959, -0.0417]]), '2.bias': tensor([0.1599])})


(tensor([[-0.4547,  0.0774, -0.3541, -0.1512],
         [-0.0430,  0.1267,  0.0890,  0.3176],
         [-0.3321,  0.3767, -0.0680, -0.1455],
         [-0.3117,  0.0956, -0.4265, -0.2773],
         [ 0.2602, -0.4928, -0.1845,  0.2387],
         [ 0.4033,  0.4966, -0.3301, -0.2480],
         [-0.3596, -0.4486, -0.0939, -0.1304],
         [-0.0832,  0.1713, -0.4213, -0.2423]]),
 tensor([ 0.2074,  0.0152, -0.3750, -0.1547,  0.2195, -0.2412,  0.2158,  0.0358]))

## [**Built-in Initialization**]

Let's begin by calling on built-in initializers.
The code below initializes all weight parameters
as Gaussian random variables
with standard deviation 0.01, while bias parameters are cleared to zero.


In [5]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0154, -0.0035, -0.0010,  0.0112]), tensor(0.))

We can also initialize all the parameters
to a given constant value (say, 1).


In [6]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

[**We can also apply different initializers for certain blocks.**]
For example, below we initialize the first layer
with the Xavier initializer
and initialize the second layer
to a constant value of 42.


In [7]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1294,  0.3904,  0.1732, -0.1723])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**Custom Initialization**]

Sometimes, the initialization methods we need
are not provided by the deep learning framework.
In the example below, we define an initializer
for any weight parameter $w$ using the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \textrm{ with probability } \frac{1}{4} \\
            0    & \textrm{ with probability } \frac{1}{2} \\
        U(-10, -5) & \textrm{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Again, we implement a `my_init` function to apply to `net`.


In [10]:
def my_init(module):
    if isinstance(module, nn.Linear):
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5 # two ranges (=10, -5) and (5, 10) has distributions

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.6293,  5.9631, -8.7696,  9.7974],
        [ 6.2627,  0.0000, -0.0000,  9.0105]], grad_fn=<SliceBackward0>)

Note that we always have the option
of setting parameters directly.


In [11]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  6.9631, -7.7696, 10.7974])

## Summary

We can initialize parameters using built-in and custom initializers.

## Exercises

Look up the online documentation for more built-in initializers.


[Discussions](https://discuss.d2l.ai/t/8090)
